In [1]:
import requests
import dask.dataframe as dd
from datetime import datetime
import json
import pandas as pd
import os
from dotenv import load_dotenv
# from delta.tables import DeltaTable

load_dotenv()

True

In [ ]:
import requests
import base64
def get_cm_access_token():
    # Credentials
    password = os.getenv("CM_API_PASSWORD", "")  # Default value for testing
    username = "cmapi"
    # Token URL
    token_url = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web/OAuth/Token"
    # Encode Basic Auth header
    credentials = f"{username}:{password}"
    encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
    # Headers and form data
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    # Token request
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        token = response.json().get("access_token")
        return token
    else:
        raise Exception(f"Failed to retrieve token: {response.status_code} - {response.text}")
    
    
# Test token retrieval
cm_access_token = get_cm_access_token()
if cm_access_token:
    print("CM token successfully retrieved")
else:
    print("Token retrieval failed")

CM token successfully retrieved


In [ ]:
BASE_URL = "https://rockpoolrac.residential.icarehealth.com.au/Tetra.Web"

In [ ]:
headers = {
    "Authorization": f"Bearer {cm_access_token}",
    "Accept": "application/json",
}

form_id = "153770"
USER_ID = "2905"


url = f"{BASE_URL}/api/AssessmentForms/GetAssessmentFormForEdit?formId={form_id}&userId={USER_ID}"
residents_bulk_sync_url = ""
# unfinished_form_url = f"{BASE_URL}/api/AssessmentForms/GetResidentUnfinishedForms"

In [ ]:
def fetch_api(url, headers):
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    json_text = resp.text
    return json_text

In [ ]:
records = []

record = fetch_api(url, headers)
if isinstance(record, dict) and "data" in record:
    records = record["data"]


# Quick sanity check:
for i, rec in enumerate(records[:5]):
    print(i, type(rec))

In [ ]:
df = pd.json_normalize(
    records,
    record_path=["items", "responses"],
    meta=[
        "formId",
        "formVersion",
        "status",
        "dateCompleted",
        ["items", "templateQuestion", "text"]
    ],
    record_prefix="resp_",
    meta_prefix="meta_"
)

# 3. Rename that nested question‐text column
df = df.rename(columns={"meta_items.templateQuestion.text": "question_text"})

print(df.dtypes)


Series([], dtype: object)


In [ ]:
print(df.head(5))

Empty DataFrame
Columns: []
Index: []


In [ ]:

# then
ddf = dd.from_pandas(df, npartitions=4)  # or read JSON directly with dask.bag/json if large

In [ ]:
ddf.head(5)  # Display first 5 rows of the Dask DataFrame

c:\Users\HarryZhan\miniconda3\envs\rockpool\Lib\site-packages\dask\dataframe\core.py:382: UserWarning: Insufficient elements for `head`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


""
